In [ ]:
1+2

# Model size

In [ ]:
import Printf

In [ ]:
function test_size()
    # model size
    @info "FC model"
    for d in [7, 10,15,20,25,30]
        Printf.@printf "%.2f\n" param_count(fc_model_fn(d)) / 1e6
    end
    @info "FC deep model"
    for d in [7, 10,15,20,25,30]
        Printf.@printf "%.2f\n" param_count(deep_fc_model_fn(d)) / 1e6
    end
    # EQ models is independent of input size
    @info "EQ model"
    Printf.@printf "%.2f\n" param_count(eq_model_fn(10)) / 1e6
    Printf.@printf "%.2f\n" param_count(deep_eq_model_fn(10)) / 1e6
end

# Old Main

In [ ]:
# setup GPU memory limit

# For 1070
g=5.0
# For 2080 Ti
# g=9.0

ENV["JULIA_CUDA_MEMORY_LIMIT"] = convert(Int, round(g * 1024 * 1024 * 1024))

In [ ]:
include("main.jl")

In [ ]:
main_CNN_sep
main_EQ_sep

In [ ]:
main_CNN_sep()

In [ ]:
main_EQ_sep()

In [ ]:
main_FC()

In [ ]:
main_CNN_ensemble()

In [ ]:
main_EQ_ensemble()

In [ ]:
main_EQ_cov()

# New version of main

- refactor the code of main1
- use seeding
- pre-gen graphs and splitting
- use cloud GPU for training

- [ ] I don't need correlation experiments anymore

In [ ]:
include("main2.jl")

In [ ]:
main2()